# A Journey into Cloud-Native Earth Observation

## Visualizing GeoZarr
In this notebook, we'll explore client-side rendered GeoZarr in Jupyter Notebooks and in the browser, using the same underlying technology. 

We'll start by connecting to a catalog of Sentinel-2 imagery, find a specific scene that catches our interest, and then—using GeoZarr and [EOxElements](https://github.com/EOX-A/EOxElements) based on the latest [OpenLayers dev release](https://www.npmjs.com/package/ol/v/10.7.1-dev.1768898466014) we can visualize an item in the browser using web components, and [ipyeoxelements](https://github.com/EOX-A/EOxElements-Jupyter) in a Python environment. We'll stream that high-resolution data directly onto an interactive map.

## Gathering Our Tools

We'll need `httpx` to communicate with the outside world (the API) and `ipyeoxelements`, our lens for viewing the geospatial data.

In [1]:
import httpx
import json
from ipyeoxelements import EOxMap, EOxLayercontrol
from ipywidgets import HBox, Layout

## Charting the Course

Here, we define our data source, the EOPF-Explorer STAC API, specifically the `sentinel-2-l2a` collection.

In [2]:
stac_endpoint = "https://api.explorer.eopf.copernicus.eu/stac/"
s3_endpoint = "https://s3.explorer.eopf.copernicus.eu/"
collection_id = "sentinel-2-l2a"

## Surveying the Landscape

First, let's take a look by querying the collection metadata, to get a sense of what kind of data is available to us.

In [3]:
collection = httpx.get(f"{stac_endpoint}collections/{collection_id}").json()
print(json.dumps(collection, indent=4))

{
    "id": "sentinel-2-l2a",
    "type": "Collection",
    "links": [
        {
            "rel": "items",
            "type": "application/geo+json",
            "href": "https://api.explorer.eopf.copernicus.eu/stac/collections/sentinel-2-l2a/items"
        },
        {
            "rel": "parent",
            "type": "application/json",
            "href": "https://api.explorer.eopf.copernicus.eu/stac/"
        },
        {
            "rel": "root",
            "type": "application/json",
            "href": "https://api.explorer.eopf.copernicus.eu/stac/"
        },
        {
            "rel": "self",
            "type": "application/json",
            "href": "https://api.explorer.eopf.copernicus.eu/stac/collections/sentinel-2-l2a"
        },
        {
            "rel": "license",
            "href": "https://sentinel.esa.int/documents/247904/690755/Sentinel_Data_Legal_Notice",
            "type": "application/pdf",
            "title": "Legal notice on the use of Copernicus Se

## Finding a Hidden Gem

Now, we dive into the catalog to find a specific scene. We'll query for a Sentinel-2 scene over Venice, Italy with minimal cloud cover to examine closely.

In [4]:
#  Venice, Italy bbox
venice_bbox = ",".join(["11.858994964808403", "44.099770918209835", "12.936097556086393", "45.10721083440407"])
items_response = httpx.get(
    f"{stac_endpoint}search",
    params={
        "collection": collection_id,
        "filter": "eo:cloud_cover<=10",
        "bbox": venice_bbox,
        "limit": 10
    }
).json()
item = items_response.get("features", [])[7]
print(f"Fetched item: {item['id']}")

Fetched item: S2B_MSIL2A_20251110T100139_N0511_R122_T32TQQ_20251110T123825


## Preparing the View

With our item identified, we need to translate it into something our map can understand. We'll initialize a GeoZarr Tile layer and add the EOxCloudless Terrain baselayer

In [5]:
def get_geozarr_store(item):
    """Construct the GeoZarr URL from the STAC item links."""
    for link in item["links"]:
        if link["rel"] == "store":
            print(f"Zarr store link: {link['href']}")
            return link["href"]


layers = []

item_id = item["id"]
zarr_url = get_geozarr_store(item)

layers = [
    {
        "type": "Tile",
        "properties": {
            "id": "terrain-light",
            "title": "Terrain Light",
        },
        "source": {
            "type": "XYZ",
            "url": "https://s2maps-tiles.eu/wmts/1.0.0/terrain-light_3857/default/g/{z}/{y}/{x}.jpeg",
            "projection": "EPSG:3857",
        },
    },
    {
        "type": "WebGLTile",
        "properties": {
            "id": f"geozarr-{item_id}",
            "title": item_id,
            "layerControlExpand": True,
        },
        "source": {
            "type": "GeoZarr",
            "url": zarr_url,
            "group": "measurements/reflectance",
            "bands": ["b04", "b03", "b02"],  # true color composite
        },
        "style": {
            "gamma": 1.5,
            "color": [
                "color",
                ["interpolate", ["linear"], ["band", 1], 0, 0, 0.5, 255],
                ["interpolate", ["linear"], ["band", 2], 0, 0, 0.5, 255],
                ["interpolate", ["linear"], ["band", 3], 0, 0, 0.5, 255],
                [
                    "case",
                    ["==", ["+", ["band", 1], ["band", 2], ["band", 3]], 0],
                    0,
                    1,
                ],
            ],
        },
    },
]

Zarr store link: https://s3.explorer.eopf.copernicus.eu/esa-zarr-sentinel-explorer-fra/tests-output/sentinel-2-l2a/S2B_MSIL2A_20251110T100139_N0511_R122_T32TQQ_20251110T123825.zarr


## The Reveal

Finally, we bring it all together. We create our map and layer control widgets and arrange them side-by-side, just like laying out a map on a table next to our field notes.

In [6]:
MAP_ID = "eopf-explorer-map"

map_widget = EOxMap(
    id=MAP_ID,
    layers=layers,
    center=[1362404,5572880],
    zoom=10,
    layout=Layout(height="600px", flex="2"),
    version="2.1.0-dev.1",
)

layer_control = EOxLayercontrol(
    for_=f"#{MAP_ID}",
    tools=["config"],
    layout=Layout(height="600px", flex="1", overflow="auto"),
)

# Display map and layer control side by side
HBox([map_widget, layer_control], layout=Layout(width="100%"))

## Replicating in the Browser

While the Python environment is great for exploration, sometimes we want to share our discoveries on the web. We can replicate this exact setup using standard HTML and the `EOxElements` web components.

To learn more read entire story at https://explorer.eopf.copernicus.eu/story?id=cloud-native-eo